In [ ]:
import pandas as pd  #libraries loading
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import re
!pip install -q wordcloud
!pip install torchmetrics
import wordcloud
from torch.utils.data import Dataset
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('omw-1.4')
from torchtext.vocab import GloVe
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.core.display import Path
from gensim.scripts.glove2word2vec import glove2word2vec
import torch
import torch.nn as nn
from torchvision import datasets 
from torchvision.transforms import ToTensor
import torch.nn.functional as F 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader , RandomSampler
import random
import os
os.environ['PYTHONHASHSEED'] = str(69)
from sklearn.metrics import classification_report, RocCurveDisplay, roc_curve, auc
from sklearn.metrics import f1_score 
from torchmetrics import ConfusionMatrix
!pip install transformers
from transformers import BertTokenizer, BertModel, BertTokenizerFast
!pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import pickle 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def randreset(s):  #making sure each run will yield consistent results
	random.seed(s)
	np.random.seed(s)
	torch.manual_seed(s)
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False
	torch.cuda.manual_seed_all(s)
	os.environ['PYTHONHASHSEED'] = str(s)
 

randreset(69)
bertmodel = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(bertmodel)

cuda


In [ ]:
pathtrain = '/content/drive/My Drive/AI2/HW4/annotated_wd_data_train.txt'     #loading data from the files that we downloaded
pathval = '/content/drive/My Drive/AI2/HW4/annotated_wd_data_valid.txt'
pathtesting = '/content/drive/My Drive/AI2/HW4/annotated_wd_data_test.txt'

dftrain = pd.read_csv(pathtrain,  sep='\t', lineterminator='\n', header=None)
dfval = pd.read_csv(pathval,  sep='\t', lineterminator='\n', header=None)
dftesting = pd.read_csv(pathtesting,  sep='\t', lineterminator='\n', header=None)


max = dftesting[3].map(lambda r: len(r.split())).max()
print(max)
max = dftrain[3].map(lambda r: len(r.split())).max()
print(max)
max = dfval[3].map(lambda r: len(r.split())).max()
print(max)

# dfval[1]
# relations = set()      #keeping track of all the entities and relations of the dataset that we are interested in
# relations.update(dftesting[1])
# relations.update(dfval[1])
# relations.update(dftrain[1])
# relations = list(relations)
# print(len(relations))   #doing this so we have a constant lexic for the other files
relations = ['R376', 'R131', 'P115', 'R176', 'P800', 'P113', 'P404', 'P509', 'P1408', 'P144', 'P59', 'R17', 'R119', 'P17', 'P264', 'R676', 'P272', 'R105', 'P105', 'P21', 'R162', 'P397', 'P1040', 'P31', 'P140', 'R279', 'P189', 'P112', 'P138', 'R1040', 'R361', 'P407', 'R179', 'R272', 'P131', 'P170', 'R58', 'P119', 'R136', 'R86', 'P376', 'R421', 'R106', 'P50', 'P1142', 'R40', 'R607', 'R112', 'R175', 'R1431', 'P641', 'P1431', 'P738', 'P607', 'P172', 'P196', 'R264', 'P413', 'P495', 'R178', 'P175', 'P57', 'R177', 'P150', 'R123', 'R170', 'R57', 'P676', 'R138', 'P156', 'R161', 'P20', 'P58', 'P19', 'P344', 'R115', 'P289', 'R149', 'R144', 'P279', 'P27', 'P421', 'R172', 'R21', 'R413', 'R641', 'R59', 'P398', 'R19', 'P403', 'P710', 'P179', 'P155', 'R344', 'P276', 'R50', 'P826', 'P53', 'P1308', 'R171', 'R404', 'P40', 'R289', 'P61', 'P136', 'P86', 'R287', 'R31', 'P123', 'P364', 'P1303', 'P287', 'P178', 'P177', 'P171', 'R1142', 'P84', 'P176', 'R509', 'P361', 'R276', 'P81', 'P737', 'P162', 'P65', 'P106', 'R1303', 'P149', 'P1029']


entities = set()
entities.update(dftrain[0])
entities.update(dfval[0])
entities.update(dftesting[0])
print(len(entities))



24
33
20
40284


# Relation Prediction

In [ ]:
class CustomDataset(Dataset): #same as in the IMDB reviews part
  def __init__(self, X, Y, tokenizer, maxlen):
    self.X = []
    self.Y = []
    self.tokenizer = tokenizer
    self.maxlen = maxlen

    for (x,y) in zip(X,Y):
      self.X.append(self.tokenizer.encode_plus(x, add_special_tokens=True, max_length=self.maxlen, padding='max_length', truncation=True, return_tensors='pt' , return_token_type_ids=False, return_attention_mask=True).to(device))
      self.Y.append(torch.tensor(y, dtype=torch.long, device=device))

  def __len__(self):
    return len(self.Y)

  def __getitem__(self, index):
    return (self.X[index]['input_ids'], self.X[index]['attention_mask'], self.Y[index])
  

In [ ]:
Xtrain = list(dftrain[3].values)   
Ytrain = list(dftrain[1].apply(lambda r: relations.index(r)))  #the target is the classification of relation out of all the possible relations in the dataset
traind = CustomDataset(Xtrain, Ytrain, tokenizer, 33)

Xval = list(dfval[3].values)
Yval = list(dfval[1].apply(lambda r: relations.index(r)))
vald = CustomDataset(Xval, Yval, tokenizer, 33)

Xtesting = list(dftesting[3].values)
Ytesting =  list(dftesting[1].apply(lambda r: relations.index(r)))
testingd = CustomDataset(Xtesting, Ytesting, tokenizer, 33)


train_dataloader = DataLoader(traind , batch_size=40, sampler=RandomSampler(traind))
val_dataloader= DataLoader(vald , batch_size=40,sampler=RandomSampler(vald))
testing_dataloader = DataLoader(testingd , batch_size=40,sampler=RandomSampler(testingd))

In [ ]:
class BERTmodel(nn.Module):
  def __init__(self, dr, H, activation):
    super(BERTmodel, self).__init__()
    self.bertlayer = BertModel.from_pretrained(bertmodel).to(device)
    self.linear = nn.Sequential(
          nn.Linear(768, H, device=device),   #768 is the output of the bert model
          activation(),
		      nn.Dropout(dr),
          nn.Linear(H,129, device=device),  #output is the distribution for the 129 possible relations, this is the only change from the IMDB part since both are basically classifications
        )

  def forward(self, ids, mask):
    out = self.bertlayer(ids, mask)
    return self.linear(out[0][:, 0, :])

  def predict(self, dataloader):
    pred = torch.tensor([], device='cpu')
    true = torch.tensor([], dtype=torch.int, device='cpu')  
    with torch.no_grad():
      self.eval()
      for (inputid, attentionmask, labels) in dataloader:
        _, y_pred = torch.max(self(inputid.squeeze(), attentionmask.squeeze()), 1)
        pred = torch.cat((pred, y_pred.cpu()))
        true = torch.cat((true, labels.squeeze().cpu()))
      return pred, true

def train(model, traindataloader, valdataloader, optimiser, learningrate, maxepochs):
  lastf1 = None
  opt = getattr(torch.optim, optimiser)(model.parameters(), lr=learningrate) 
  lossfunction = nn.CrossEntropyLoss()
  for epoch in range(maxepochs):
    model.train()
    t_losses = []
    for (inputid, attentionmask, labels) in traindataloader:
      opt.zero_grad()
      ypred = model(inputid.squeeze(), attentionmask.squeeze())
      loss = lossfunction(ypred, labels.squeeze())
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      opt.step()

    pred, true = model.predict(valdataloader)
    f1 = f1_score(true, pred, average='weighted')
    print(f"for epoch: {epoch} we got val f1 score = {f1}")

    if lastf1 is None:    #stopping early if validation f1 does not improve
      lastf1 = f1
    else: 
      if lastf1 >= f1:
        break
      else:
        lastf1 = f1
  
  return model



In [ ]:
model = BERTmodel(0.1, 54, nn.CELU)  #same as the IMDB part since it works well
model = train(model, train_dataloader, val_dataloader, "AdamW", 2.2e-05, 4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


for epoch: 0 we got val f1 score = 0.9414160051247691
for epoch: 1 we got val f1 score = 0.9609732365431753
for epoch: 2 we got val f1 score = 0.963538419023227
for epoch: 3 we got val f1 score = 0.9603346150752379


In [ ]:
pred, true = model.predict(testing_dataloader)
print(f"test f1 score is {f1_score(true, pred, average='weighted')}")
print(classification_report(true, pred))


test f1 score is 0.958067331213427
              precision    recall  f1-score   support

           1       0.96      0.93      0.95        29
           2       1.00      1.00      1.00         7
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         7
           6       1.00      0.95      0.97        57
           7       0.98      0.99      0.99       116
           8       1.00      0.92      0.96        13
           9       0.71      1.00      0.83         5
          10       0.43      1.00      0.60         3
          11       0.91      0.95      0.93        21
          12       1.00      1.00      1.00         8
          13       0.96      0.79      0.87        67
          14       0.97      0.96      0.96       157
          15       1.00      1.00      1.00        17
          16       0.90      0.92      0.91        39
          17       1.00      1.00      1.00         3
          18       0.83      0.91      0.87   

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/AI2/HW4/relationpredictor.pt')

# Span Prediction


In [ ]:
# sparql = SPARQLWrapper("https://query.wikidata.org/sparql")     #doing the required queries, saving the results since there are limits
# import time
# index = 0
# # stops = [item for item in range(1000, 40000, 1000)]
# index = 39965
# entities = list(entities)
# for ent in entities[39965:]:
#   sparql.setQuery("""
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
# PREFIX wd: <http://www.wikidata.org/entity/> 
# SELECT  *
# WHERE {
#         wd:%s rdfs:label ?label .
#         FILTER (langMatches( lang(?label), "EN" ) )
#       }
# """ %(ent))
  
#   sparql.setReturnFormat(JSON)
#   results = sparql.query().convert()

#   results_df = pd.json_normalize(results['results']['bindings'])
#   if set(["label.value"]).issubset(results_df):
#     # print(results_df["label.value"][0])
#     # entitylabels[ent] = results_df["label.value"][0]
#     f = open("/content/drive/My Drive/AI2/HW4/dump.txt", "a")
#     f.write(f'{ent}  {results_df["label.value"][0]} \n')
#     f.close()
#   print(index)

#   # if index in stops:
#   #   time.sleep(305)
#   index+=1



In [ ]:
# f = open("/content/drive/My Drive/AI2/HW4/dump.txt", "r")
# entitylabels = {}

# for i in range(39925):	
#   line = f.readline()
#   tok = line.split()
#   entitylabels[tok[0]] = tok[1]

# with open('/content/drive/My Drive/AI2/HW4/saved_dictionary.pkl', 'wb') as f:
#   pickle.dump(entitylabels, f)

In [ ]:
with open('/content/drive/My Drive/AI2/HW4/saved_dictionary.pkl', 'rb') as f:   #loading the dictionary with the entities and their label
  entitylabels = pickle.load(f)

# print(entitylabels)

In [ ]:
def createspantarget(entitylabels, text, labels):  #output 33 size vectors with 0-1
  spans = []
  for (x,label) in zip(text, labels):
    tokens = x.split()

    if label not in entitylabels:    #if we cant find the label for whatever reason we pass an all zeroes vector
      spans.append([0] * 33)
      continue

    target = entitylabels[label]
    targettokens = target.split()
    if(x.rfind(target) == -1):  #if the substring is not in the question we wont be able to find it so we move on with a zeor vector
      spans.append([0] * 33)
    else:
      l = [0] * 33
      for i,w in enumerate(tokens):
        if w in targettokens:    #whenever we find a part of teh question on the substring, we put a 1 in that place
          l[i] = 1;
      spans.append(l)

  return spans

In [ ]:
class SpanCustomDataset(Dataset):
  def __init__(self, X, Y, tokenizer, maxlen):
    self.X = []
    self.Y = []
    self.tokenizer = tokenizer
    self.maxlen = maxlen

    for (x,y) in zip(X,Y):
      if 1 in y:                #only if we managed to find the span we keep the question on the dataset
        self.X.append(self.tokenizer.encode_plus(x, add_special_tokens=True, max_length=self.maxlen, padding='max_length', truncation=True, return_tensors='pt' , return_token_type_ids=False, return_attention_mask=True).to(device))
        self.Y.append(torch.Tensor(y).to(device))


  def __len__(self):
    return len(self.Y)

  def __getitem__(self, index):
    return (self.X[index]['input_ids'], self.X[index]['attention_mask'], self.Y[index])

In [ ]:
Xtrain = list(dftrain[3].values)
Ytrain = createspantarget(entitylabels, Xtrain, dftrain[0])
traind = SpanCustomDataset(Xtrain, Ytrain, tokenizer, 33)

Xval = list(dfval[3].values)
Yval = createspantarget(entitylabels, Xval, dfval[0])
vald = SpanCustomDataset(Xval, Yval, tokenizer, 33)

Xtesting = list(dftesting[3].values)
Ytesting = createspantarget(entitylabels, Xtesting, dftesting[0])
testingd = SpanCustomDataset(Xtesting, Ytesting, tokenizer, 33)

train_dataloader = DataLoader(traind , batch_size=40, sampler=RandomSampler(traind))
val_dataloader= DataLoader(vald , batch_size=40,sampler=RandomSampler(vald))
testing_dataloader = DataLoader(testingd , batch_size=40,sampler=RandomSampler(testingd))

In [ ]:
class BERTmodelforSpan(nn.Module):
  def __init__(self, dr, H, activation):
    super(BERTmodelforSpan, self).__init__()
    self.bertlayer = BertModel.from_pretrained(bertmodel).to(device)
    self.linear = nn.Sequential(
          nn.Linear(768, H, device=device),   #768 is the output of the bert model
          activation(),
		      nn.Dropout(dr),
          nn.Linear(H,33, device=device),   #output is 33 for the 33 places on the vector for the span
        )

  def forward(self, ids, mask):
    out = self.bertlayer(ids, mask)
    return torch.sigmoid(self.linear(out[0][:, 0, :]))

  def predict(self, dataloader):
    pred = torch.tensor([], device='cpu')
    true = torch.tensor([], dtype=torch.int, device='cpu')  #using tensors so I can control that they are in the cpu in order for sklearn's functions to not complain
    with torch.no_grad():
      self.eval()
      for (inputid, attentionmask, labels) in dataloader:
        y_pred = torch.round(self(inputid.squeeze(), attentionmask.squeeze()))             
        pred = torch.cat((pred, y_pred.cpu()))
        true = torch.cat((true, labels.squeeze().cpu()))
      return pred, true

def train(model, traindataloader, valdataloader, optimiser, learningrate, maxepochs):
  lastf1 = None
  opt = getattr(torch.optim, optimiser)(model.parameters(), lr=learningrate) 
  lossfunction = nn.BCELoss()
  for epoch in range(maxepochs):
    model.train()
    t_losses = []
    for (inputid, attentionmask, labels) in traindataloader:
      opt.zero_grad()
      ypred = model(inputid.squeeze(), attentionmask.squeeze())
      loss = lossfunction(ypred, labels.squeeze())
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      opt.step()

    pred, true = model.predict(valdataloader)
    f1 = f1_score(true, pred, average='weighted')
    print(f"for epoch: {epoch} we got val f1 score = {f1}")

    if lastf1 is None:    #stopping early if validation f1 does not improve
      lastf1 = f1
    else: 
      if lastf1 >= f1:
        break
      else:
        lastf1 = f1
  
  return model

In [ ]:
model = BERTmodelforSpan(0.1, 54, nn.CELU)
model = train(model, train_dataloader, val_dataloader, "AdamW", 2.2e-05, 15)
pred, true = model.predict(val_dataloader)
print(f"test f1 score is {f1_score(true, pred, average='weighted')}")
print(classification_report(true, pred))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labe

for epoch: 0 we got val f1 score = 0.0


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


for epoch: 1 we got val f1 score = 0.054613288824525666


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


for epoch: 9 we got val f1 score = 0.8841512207243134


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


for epoch: 10 we got val f1 score = 0.8697744865153243
test f1 score is 0.8697744865153243
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       1.00      0.78      0.88        27
           2       0.99      0.97      0.98       127
           3       0.98      0.99      0.98       140
           4       0.96      0.91      0.93        78
           5       0.88      0.87      0.87        92
           6       0.92      0.89      0.90        89
           7       0.86      0.56      0.68        32
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         7
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classifi

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/AI2/HW4/spandetector.pt')